<a href="https://colab.research.google.com/github/clashleyuncc/Intro-To-ML/blob/main/HW7P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# P2
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

import torch.nn.functional as F

class ResBlock(nn.Module):
  def __init__(self, n_chans):#16
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                          padding=1, bias=False)
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight,
                                  nonlinearity='relu')
    torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = self.batch_norm(out)
    out = torch.relu(out)
    return out + x

class ResNet10(nn.Module):
  def __init__(self, n_chans1=32, n_blocks=10):
    super().__init__()
    self.n_chans1 = n_chans1
    self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
    self.resblocks = nn.Sequential(
        *(n_blocks * [ResBlock(n_chans=n_chans1)]))
    self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
    out = self.resblocks(out)
    out = F.max_pool2d(out, 2)
    out = out.view(-1, 8 * 8 * self.n_chans1)
    out = torch.relu(self.fc1(out))
    out = self.fc2(out)
    return out

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    if epoch == 1:
      print('Esimated run time for {} Epoch: {} minutes'.format(
            n_epochs, round((time.time()-start_time)*n_epochs/60,3)))
    if epoch == 1 or epoch % 10 == 0:
      print('{} Seconds, Epoch {}, Training Loss {}'.format(
          time.time()-start_time, epoch,
          loss_train / len(train_loader)))

from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                  transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                  transform=transforms.ToTensor())

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

tensor_cifar10 = transformed_cifar10

model = ResNet10().to(device=device)
optimizer = optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0

    with torch.no_grad():
      for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print("Accuracy {}: {:.2f}".format(name , correct/total))

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64,
                                         shuffle=False)

validate(model, train_loader, val_loader)

print(f"Training Time: {time.time()-start_time} seconds")
print(f"Approximatly: {round((time.time()-start_time)/60,3)} minutes")


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Esimated run time for 200 Epoch: 65.257 minutes
19.57721781730652 Seconds, Epoch 1, Training Loss 1.7372073159193444
161.4488546848297 Seconds, Epoch 10, Training Loss 0.919807996324566
313.96229243278503 Seconds, Epoch 20, Training Loss 0.7063378002637487
472.88633370399475 Seconds, Epoch 30, Training Loss 0.5731973091278539
632.3554894924164 Seconds, Epoch 40, Training Loss 0.4676260417303466
790.1446046829224 Seconds, Epoch 50, Training Loss 0.375128383755379
948.7587575912476 Seconds, Epoch 60, Training Loss 0.301872299569647
1104.570143699646 Seconds, Epoch 70, Training Loss 0.23517241163174515
1258.865888595581 Seconds, Epoch 80, Training Loss 0.18812784272939195
1411.115600824356 Seconds, Epoch 90, Training Loss 0.1594656245673404
1559.4171395301819 Seconds, Epoch 100, Training Loss 0.13217191253205204
1711.2565841674805 Seconds, Epoch 110, Training Loss 0.09023544567582362
1871.234411